In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
torch.manual_seed(42)  # for reproducibility

In [4]:
df1 = pd.read_csv('/home/darshan39/Downloads/fashionmnist/fashion-mnist_train.csv')
df2 = pd.read_csv('/home/darshan39/Downloads/fashionmnist/fashion-mnist_test.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 419.2 MB


In [6]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
)

In [8]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [9]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(56000, 784), dtype=float32)

In [10]:
class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [11]:
train_dataset = FashionMNISTDataset(x_train, y_train)

In [12]:
train_dataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000,
         0.0510, 0.4627, 0.1843, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [13]:
test_dataset = FashionMNISTDataset(x_test, y_test)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [15]:
class myNN(nn.Module):
    def __init__(self, num_features):
        super(myNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        return self.model(x)

In [16]:
learning_rate = 0.1
num_epochs = 100

In [17]:
model = myNN(x_train.shape[1])
model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [18]:
for epoch in range(num_epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:


        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        # Forward pass
        outputs = model(batch_features)


        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()


        loss.backward()


        optimizer.step()

        total_epoch_loss += loss.item()
    
    avg_epoch_loss = total_epoch_loss / len(train_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_epoch_loss:.4f}")
    

Epoch [1/100], Loss: 0.6093
Epoch [2/100], Loss: 0.4797
Epoch [3/100], Loss: 0.4424
Epoch [4/100], Loss: 0.4237
Epoch [5/100], Loss: 0.4085
Epoch [6/100], Loss: 0.3961
Epoch [7/100], Loss: 0.3863
Epoch [8/100], Loss: 0.3777
Epoch [9/100], Loss: 0.3696
Epoch [10/100], Loss: 0.3607
Epoch [11/100], Loss: 0.3570
Epoch [12/100], Loss: 0.3515
Epoch [13/100], Loss: 0.3466
Epoch [14/100], Loss: 0.3451
Epoch [15/100], Loss: 0.3405
Epoch [16/100], Loss: 0.3380
Epoch [17/100], Loss: 0.3356
Epoch [18/100], Loss: 0.3361
Epoch [19/100], Loss: 0.3306
Epoch [20/100], Loss: 0.3303
Epoch [21/100], Loss: 0.3248
Epoch [22/100], Loss: 0.3253
Epoch [23/100], Loss: 0.3188
Epoch [24/100], Loss: 0.3157
Epoch [25/100], Loss: 0.3157
Epoch [26/100], Loss: 0.3150
Epoch [27/100], Loss: 0.3103
Epoch [28/100], Loss: 0.3111
Epoch [29/100], Loss: 0.3060
Epoch [30/100], Loss: 0.3082
Epoch [31/100], Loss: 0.3031
Epoch [32/100], Loss: 0.3045
Epoch [33/100], Loss: 0.3014
Epoch [34/100], Loss: 0.3023
Epoch [35/100], Loss: 0

In [19]:
# Evaluation on train set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_features, batch_labels in train_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total
    print(accuracy)

0.9411964285714286


In [20]:
# Evaluation on test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_features, batch_labels in test_loader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

    accuracy = correct / total
    print(accuracy)

0.8925714285714286
